In [1]:
# This file plays the mpc motion only in viz
%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pinocchio as pin
import crocoddyl
from matplotlib import pyplot as plt

from py_biconvex_mpc.motion_planner.cpp_biconvex import BiConvexMP
from inverse_kinematics_cpp import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator
from robot_properties_solo.config import Solo12Config

import raisimpy as raisim
import subprocess
from py_biconvex_mpc.bullet_utils.solo_mpc_env import Solo12Env

from mpc_cartwheel import generate_plan

solo import 


pybullet build time: Jul  2 2021 21:43:18


In [2]:
robot = Solo12Config.buildRobotWrapper()
rmodel = robot.model
rdata = robot.data

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

pin_robot = Solo12Config.buildRobotWrapper()
pin_robot = Solo12Config.buildRobotWrapper()
urdf = Solo12Config.urdf_path


q0 = np.array(Solo12Config.initial_configuration)
v0 = pin.utils.zero(pin_robot.model.nv)
x0 = np.concatenate([q0, pin.utils.zero(pin_robot.model.nv)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
viz.viewer.jupyter_cell()

In [116]:
sim_t = 0.0
sim_dt = .001
index = 0
pln_ctr = 0
plan_freq = 0.5 # sec
update_time = 0.0 # sec (time of lag)
lag = int(update_time/sim_dt)

q = q0
v = v0

for o in range(10):
    contact_configuration = np.array([1,1,1,1])
    xs, us, f = generate_plan(q, v, sim_t)
    xs = xs[lag:]
    us = us[lag:]
    f = f[lag:]

    time.sleep(0.001)
    for ind in range(int(plan_freq/sim_dt)):
        viz.display(xs[ind][:robot.model.nq])

    q = xs[int(plan_freq/sim_dt)-1][0:pin_robot.model.nq]
    v = xs[int(plan_freq/sim_dt)-1][pin_robot.model.nq:]

    sim_t += plan_freq


Maximum iterations reached 
Final norm: 0.0212427
dyn : 0.07573986053466797
ik : 0.04758858680725098
total : 0.12333059310913086


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

Maximum iterations reached 
Final norm: 0.00713172
dyn : 0.05692911148071289
ik : 0.013010978698730469
total : 0.06994223594665527


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

...

Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.000918377
dyn : 0.020866870880126953
ik : 0.0442662239074707
total : 0.06513619422912598


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit

breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.000918775
dyn : 0.01979660987854004
ik : 0.03993034362792969
total : 0.059729576110839844
breaking outer loop due to norm ...
Maximum iterations reached 
Final norm: 0.000918911
dyn : 0.02002739906311035
ik : 0.0421147346496582
total : 0.06214547157287598


Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition with CostModelResidual
Deprecated CostModelCoMPosition: Use ResidualModelCoMPosition wit